# generate fake link with time info
## execute next block first 

In [13]:
# randomly sample test link
import numpy as np
from tqdm import tqdm
from os.path import join
np.random.seed(1337)
def get_node_set(path):
    # training data
    edges_unordered = np.genfromtxt(path,
                                    dtype=np.int32)
    id_set = set(edges_unordered.flatten().tolist())
    return id_set

data_path = join('./','kaggle')
# training data
train_node_set = get_node_set(join(data_path,'t3-train.txt'))
test_node_set = get_node_set(join(data_path,'t3-test.txt')).difference(train_node_set)
node_set = set.union(train_node_set, test_node_set)
idx_map = {k:i for i,k in enumerate(list(node_set))}
N = len(node_set)
adj_mat = np.zeros([N,N], dtype=np.uint8)
W = []

links = np.genfromtxt(join(data_path,'t3-train.txt'), dtype=np.int32)
for i in range(links.shape[0]):
    src, dst = links[i].tolist()
    _,src_weeks =  xml_id_map[src]
    _,dst_weeks =  xml_id_map[dst]
    
    weeks_distance = max(src_weeks - dst_weeks, 1.)
    W.append(weeks_distance)
    adj_mat[idx_map[src], idx_map[dst]] = 1

out_degree = np.sum(adj_mat, axis=1).flatten()

means, std = np.mean(out_degree), np.std(out_degree)
w_mean, w_std = np.mean(W), np.std(W)
print w_mean, w_std

rev_map = {v:k for k,v in idx_map.items()}
total_num = links.shape[0] + int(1.2*means * len(test_node_set))
c = 0

with tqdm(total=total_num) as pbar:
    with open(join(data_path,'t3-fake.txt'), 'w') as f:
        for i in range(links.shape[0]):
            src, dst = links[i].tolist()
            s = '%d %d\n' % (src, dst)
            f.write(s)
            pbar.update(1)
        train_node_list = list(train_node_set)
        for node_id in list(test_node_set):
            _,src_weeks =  xml_id_map[node_id]
                
            i = idx_map[node_id]
            d = int(np.round(np.random.normal(means, std)))
            d = max(1, d)
            
            for j in range(d):
                
                idx = np.random.randint(len(train_node_list))
                dst = idx_map[train_node_list[idx]]
                _,dst_weeks =  xml_id_map[train_node_list[idx]]
                w_d = src_weeks-dst_weeks
                k = 0
                while adj_mat[i, dst] == 1 or dst == i or src_weeks < dst_weeks or abs(abs(w_d)- w_mean) > 2*w_std:
                    idx = np.random.randint(len(train_node_list))
                    dst = idx_map[train_node_list[idx]]
                    _,dst_weeks =  xml_id_map[train_node_list[idx]]
                    w_d = src_weeks-dst_weeks
#                     print '%.2f -> %.2f' %(src_weeks, dst_weeks),(adj_mat[i, dst] == 1 or dst == i), src_weeks < dst_weeks
                    k+=1
                    if k > 10000 and src_weeks < avg_w/2:
                        c +=1
                        break
#                         print k,node_id, train_node_list[idx],'%.2f -> %.2f' %(src_weeks, dst_weeks), src_weeks < dst_weeks,(adj_mat[i, dst] == 1 or dst == i), abs(abs(w_d)- w_mean) > 2*w_std
                if k > 10000 and src_weeks < avg_w/2:
                    pbar.update(1)
                    continue 
                if node_id == 17493:
                    print node_id, dst, src_weeks, dst_weeks
                adj_mat[i, dst] = 1
                s = '%d %d\n' % (rev_map[i], rev_map[dst])
                f.write(s)
            
                pbar.update(1)
#             print c,len (test_node_set)
    
print 'done', np.sum(adj_mat)
print c, len(test_node_set)
    

 60%|█████▉    | 64710/108630 [00:00<00:00, 647099.07it/s]

[95.03980042] [87.40132677]


110594it [00:01, 108481.73it/s]                           


done 110585
9 3411


# process text info and time info

In [1]:
import keras
from os.path import join
import os
from bs4 import BeautifulSoup as BS
from constants import MAX_SEQUENCE_LENGTH, MAX_NUM_WORDS, EMBEDDING_DIM
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from datetime import datetime
from tqdm import tqdm
import re

np.random.seed(1337)
def replace_timezone_fn(s):
    return re.sub('(\d){2}:(\d){2}:(\d){2} [A-Z]{3}', _sub_tz, s)
def _sub_tz(matched):
    return re.sub('[A-Za-z]{3}', 'GMT', matched.group(0))
     
def retrive_date(pat_obj_1, pat_obj_2, pat_obj_3, s):
    s = s.lower()
    
    match_1 = pat_obj_1.search(s)
    match_2 = None
    match_3 = None
    
    if match_1 is None:
        match_2 = pat_obj_2.search(s)
    if match_2 is None:
        match_3 = pat_obj_3.search(s)
    assert (match_1 is not None) | (match_2 is not None) | (match_3 is not None)
    
        
    if match_1 is not None:
        days, months, years = match_1.groups()
        date_str = '%s %s %s' % (days, months, years)
    if match_2 is not None:
        days, months, years = match_2.groups()
        date_str = '%s %s 19%s' % (days, months, years)
    if match_3 is not None:
        months, days, years = match_3.groups()
        date_str = '%s %s %s' % (days, months, years)
    
    if len(months) > 3:
        return datetime.strptime(date_str, '%d %B %Y')
    else:
        return datetime.strptime(date_str, '%d %b %Y')
    

def prep_date(date):
#     min date in training set 1991-12-31
    secs = (date - datetime.strptime('1990-9-9', '%Y-%m-%d')).total_seconds()
    weeks = np.round(secs // (60*60*24*7))
    return weeks

def quote_title_date_abstract(xml_path):
    with open(xml_path, 'r') as f:
        data = f.read()
    soup = BS(data)
    title, abstract, date = soup.find('title').text, soup.find('abstract').text, soup.find('date').text
    
    return title.strip(), abstract.strip(), date.strip()

# text preprocessing
data_path = join('./','kaggle/')
xml_dir = join(data_path, 't3-doc')
xml_list = [f for f in os.listdir(xml_dir) if f.endswith('.xml')]
# print(len(xml_list))


texts = []
weeks = []
c = 0
# 
month_pattern = '(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|january|february|march|april|may|june|july|august|september|october|november|december)'
pat_obj_1 = re.compile('([0-9]+) %s ([0-9]{4}|20[0-9]{3})' % month_pattern)
pat_obj_2 = re.compile('([0-9]+) %s (9[0-9])' % month_pattern)
pat_obj_3 = re.compile('%s ([0-9]+) .* ([0-9]{4}|20[0-9]{3})' % month_pattern)
# 

with tqdm(total=len(xml_list)) as pbar:
    for xml in xml_list:
        
        
        path = join(xml_dir,xml)
        title, abstract, date_str = quote_title_date_abstract(path)
        text = title + '' + abstract
#         date special case
#         01/01/93 13:35:33 GMT+0100 12330.xml
        if xml == '12330.xml':
            date_str = '1 Jan 93'

        try :
            date = retrive_date(pat_obj_1, pat_obj_2, pat_obj_3, date_str)
        except AssertionError:
            print date_str, xml
            c+=1 
            continue
        texts.append(text)
        weeks.append(prep_date(date))
        pbar.update(1)
print('read all %d xml files.' % len(xml_list))
print 'error count %d' % c
min_w, avg_w, std_w = (min(weeks), np.mean(weeks), np.std(weeks))
print 'min weeks in training set : %.0f, avg weeks : %.0f' % (min_w, avg_w)
tokenizer = keras.preprocessing.text.Tokenizer(num_words=MAX_NUM_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~ ',
                                   lower=True, split=' ', char_level=False, oov_token=None)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
sequences = tokenizer.texts_to_sequences(texts)
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
xml_id_map = {}
for i,xml in enumerate(xml_list):
    node_id = int(xml.replace('.xml',''))
    w = float(weeks[i] - avg_w)/std_w 
    w = np.array([w,])
    xml_id_map[node_id] = (data[i,:], w)


print('Preparing embedding matrix.')
embeddings_index = {}
# with open(os.path.join('./','glove', 'glove.6B.%dd.txt' % EMBEDDING_DIM), 'r', encoding='utf8') as f:
with open(os.path.join('./','glove', 'glove.6B.%dd.txt' % EMBEDDING_DIM), 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print('done')

Using TensorFlow backend.
100%|██████████| 17500/17500 [00:07<00:00, 2289.00it/s]


read all 17500 xml files.
error count 0
min weeks in training set : 68, avg weeks : 396
Found 82615 unique tokens.
Preparing embedding matrix.
done


In [2]:
# emb
buf = np.genfromtxt('./t3.emb', skip_header=1, dtype=np.float32)
nodes = buf[:,0].astype(np.int32)
emb = buf[:,1:]

node_emb_dict = {}
for i in range(emb.shape[0]):
    node_id = nodes[i]
    x = emb[i,:]
    node_emb_dict[node_id] = x


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.init import xavier_normal_
from constants import D_MODEL, STACKED_NUM,DK, DV, H, P_DROP, D_FF, MAX_SEQUENCE_LENGTH, MAX_NUM_WORDS, EMBEDDING_DIM
# environment
with_gpu = torch.cuda.is_available()
# with_gpu = False
device = torch.device("cuda:0" if with_gpu else "cpu")

def positional_encoding(pos):
    assert D_MODEL % 2 == 0
    pos = torch.tensor(pos, dtype=torch.float32, requires_grad=False)
    pe = torch.zeros([1,D_MODEL], dtype=torch.float32, requires_grad=False)
    for i in range(D_MODEL//2):
        a = torch.tensor(10000, dtype=torch.float32, requires_grad=False)
        b = torch.tensor(2.*i/float(D_MODEL), dtype=torch.float32, requires_grad=False)
        c = pos / torch.pow(a, b)
        pe[0, 2*i] = torch.sin(c)
        pe[0, 2*i+1] = torch.cos(c)
    return pe
def get_pos_mat(length):
    if length > MAX_SEQUENCE_LENGTH:
        print('sequence length reach PE_MAT_CACHE. %d ' % length)
        ret = torch.cat([positional_encoding(i) for i in range(length)], dim=0).to(device)
        ret.requires_grad = False
        global PE_CACHE_MATRIX
        PE_CACHE_MATRIX = ret
        return ret
    else:
        return PE_CACHE_MATRIX[:length]
    
PE_CACHE_MATRIX = torch.cat([positional_encoding(i) for i in range(0,MAX_SEQUENCE_LENGTH)], dim=0).to(device)
PE_CACHE_MATRIX.requires_grad = False

# construct neuron network

def scaled_dot_attention(Q, K, V, mask=None):
    assert Q.size()[-1] == K.size()[-1]
    dk = torch.tensor(K.size()[-1], dtype=torch.float32, requires_grad=False).to(device)
    out = torch.matmul(Q,K.t()) / torch.sqrt(dk) 
    if mask is not None:
        out = out.masked_fill_(mask, -float('inf'))
        
    return torch.matmul(F.softmax(out, dim=-1), V)
                            
class Transformer(nn.Module):

    def __init__(self, layer_num, dk, dv, dm, h, emb_matrix):
        super(Transformer, self).__init__()
        
        self.emb = Word_Embedding(emb_matrix)
        
        self.emb_drop = nn.Dropout(P_DROP)
        
        self.encoder = Stack_Encoder(layer_num, dk, dv, dm, h)
        self.decoder = Stack_Decoder(layer_num, dk, dv, dm, h)
        self.summary_decoder = Stack_Decoder(2, dk, dv, dm, h)
        
        self.summary_weight = nn.Parameter(torch.FloatTensor(1, dm))
        torch.nn.init.xavier_uniform_(self.summary_weight)
        
#         self.q_weeks_linear = nn.Linear(1, dm//2)
#         self.k_weeks_linear = nn.Linear(1, dm//2)
        self.output_linear = nn.Linear(3*dm+2, dm)
        self.output_linear2 = nn.Linear(dm, dm)
        self.output_linear3 = nn.Linear(dm, 1)
        

    def forward(self, Q, K, Q_fea, K_fea, Q_w, K_w):
        
#         encoder
        K = self.emb(K)
#         print(K.size(), get_pos_mat(MAX_SEQUENCE_LENGTH).size())
        K = K + get_pos_mat(MAX_SEQUENCE_LENGTH)
        K = self.emb_drop(K)
        
        en_out = self.encoder(K)
        
#         decoder
        Q = self.emb(Q)
        seq_len, d = Q.size()
        
        Q = Q + get_pos_mat(MAX_SEQUENCE_LENGTH)
        Q = self.emb_drop(Q)
        
        de_out = self.decoder(Q, en_out)
        summary = self.summary_decoder(self.summary_weight, de_out)

#         q_w_out = F.selu(self.q_weeks_linear(Q_w))
#         k_w_out = F.selu(self.k_weeks_linear(K_w))
        
        
#         x = torch.cat([summary, Q_fea.view([1,-1]), K_fea.view([1,-1]), q_w_out.view([1,-1]), k_w_out.view([1,-1])], dim=-1)
        x = torch.cat([summary, Q_fea.view([1,-1]), K_fea.view([1,-1]), Q_w.view([1,-1]), K_w.view([1,-1])], dim=-1)
        out = self.output_linear(x)
        
        out = self.output_linear2(F.selu(out))
        out = self.output_linear3(F.selu(out))
        
        out = torch.sigmoid(out)
        

        return out

class Word_Embedding(nn.Module):
    def __init__(self, emb_matrix):
        super(Word_Embedding, self).__init__()
        self.emb = nn.Embedding(MAX_NUM_WORDS, EMBEDDING_DIM, padding_idx=0)
        self.emb.weight = nn.parameter.Parameter(torch.FloatTensor(emb_matrix))
        self.emb.weight.requires_grad_(False)
        
        self.linear = nn.Linear(EMBEDDING_DIM, D_MODEL, bias=False)
        


    def forward(self, x):
        x = self.emb(x)
        x = self.linear(x)
        return x
    
class Stack_Encoder(nn.Module):
    """
    Stacked Encoder
    """
    def __init__(self, layer_num, dk, dv, dm, h):
        super(Stack_Encoder, self).__init__()
        self.encoders = nn.ModuleList([Encoder(dk, dv, dm, h) for i in range(layer_num)])

    def forward(self, K):
        # ModuleList can act as an iterable, or be indexed using ints
        for lay in self.encoders:
            K = lay(K)
        return K               
class Encoder(nn.Module):
    def __init__(self, dk, dv, dm, h):
        super(Encoder, self).__init__()
#         attention residual block
        self.multi_head_attention_layer = Multi_Head_attention_layer(dk, dv, dm, h)
        self.attention_norm_lay = nn.LayerNorm([dm,])
        self.att_drop = nn.Dropout(P_DROP)
#         feed forward residual block
        self.fcn = PositionwiseFeedForward(D_MODEL, D_FF)
        self.linear_drop = nn.Dropout(P_DROP)
        self.ff_norm_lay = nn.LayerNorm([dm, ])
        

    def forward(self, K):
#         attention
        attention_out = self.multi_head_attention_layer(K, K, K)
        attention_out = self.att_drop(attention_out)
        att_out = self.attention_norm_lay(K + attention_out)
#         feed forward
        linear_out = self.fcn(att_out)
        linear_out = self.linear_drop(linear_out)
        out = self.ff_norm_lay(att_out + linear_out)
        out = att_out + linear_out
    
        return out
class Stack_Decoder(nn.Module):
    """
    Stacked Encoder
    """
    def __init__(self, layer_num, dk, dv, dm, h):
        super(Stack_Decoder, self).__init__()
        self.decoders = nn.ModuleList([Decoder(dk, dv, dm, h) for i in range(layer_num)])
        
        
    def forward(self, Q, encoder_out):
        # ModuleList can act as an iterable, or be indexed using ints
        Q_len, d = Q.size()
        for lay in self.decoders:
            Q = lay(Q, encoder_out, mask=None)
        return Q           

class Decoder(nn.Module):
    def __init__(self, dk, dv, dm, h):
        super(Decoder, self).__init__()
#         query attention residual block
        self.Q_attention_lay = Multi_Head_attention_layer(dk, dv, dm, h)
        self.Q_attention_norm_lay = nn.LayerNorm([dm, ])
        self.Q_att_drop = nn.Dropout(P_DROP)
    
#         query key attention residual block
        self.QK_attention_lay = Multi_Head_attention_layer(dk, dv, dm, h)
        self.QK_attention_norm_lay = nn.LayerNorm([dm, ])
        self.QK_att_drop = nn.Dropout(P_DROP)
        
    
#         feed forward residual block
        self.fcn = PositionwiseFeedForward(D_MODEL, D_FF)
        self.ff_norm_lay = nn.LayerNorm([dm, ])
        self.linear_drop = nn.Dropout(P_DROP)
        

    def forward(self, Q, encoder_out, mask):
#         query attention
        Q_attention_out = self.Q_attention_lay(Q, Q, Q, mask)
        Q_attention_out = self.Q_att_drop(Q_attention_out)
        Q_att_out = self.Q_attention_norm_lay(Q + Q_attention_out)
#         query key attention
        QK_attention_out = self.QK_attention_lay(Q_att_out, encoder_out, encoder_out)
        QK_attention_out = self.QK_att_drop(QK_attention_out)
        QK_att_out = self.QK_attention_norm_lay(Q_att_out + QK_attention_out)
        
#         feed forward
        linear_out = self.fcn(QK_att_out)
        out = self.ff_norm_lay(QK_att_out + linear_out)
        return out

class Multi_Head_attention_layer(nn.Module):
    def __init__(self, dk, dv, dm, h):
        super(Multi_Head_attention_layer, self).__init__()
        self.Q_linears = nn.ModuleList([nn.Linear(dm, dk) for i in range(h)])
        self.K_linears = nn.ModuleList([nn.Linear(dm, dk) for i in range(h)])
        self.V_linears = nn.ModuleList([nn.Linear(dm, dv) for i in range(h)])
        self.output_linear = nn.Linear(h*dv, dm)
                            

    def forward(self, Q_input, K_input, V_input, mask=None):
        buf = []
        for Q_linear, K_linear, V_linear in zip(self.Q_linears, self.K_linears, self.V_linears):
            Q = Q_linear(Q_input)
            K = K_linear(K_input)
            V = V_linear(V_input)
            buf.append(scaled_dot_attention(Q, K, V, mask))
            
        buf = torch.cat(buf,dim=-1)
        out = self.output_linear(buf)
        
        return out      
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionwiseFeedForward, self).__init__()
        self.cnn1 = nn.Conv1d(d_model, d_ff, 1)
        self.cnn2 = nn.Conv1d(d_ff, d_model, 1)
                            

    def forward(self, x):
        seq_len,_ = x.size()
        x = x.unsqueeze(0)
        x = x.permute(0,2,1)
        x = self.cnn1(x)
        x = F.relu(x)
        x = self.cnn2(x)
        x = x.permute(0,2,1)
        x = x.squeeze(0)
        
        return x      
    
# encoder = Stack_Encoder(6, 64,64,20,8)
# # print net
Q = torch.randint(10000,[MAX_SEQUENCE_LENGTH,], dtype=torch.long).to(device)
V = torch.randint(10000,[MAX_SEQUENCE_LENGTH,], dtype=torch.long).to(device)
Q_fea = torch.rand([D_MODEL,]).to(device)
K_fea = torch.rand([D_MODEL,]).to(device)
Q_w = torch.rand([1,]).to(device)
K_w = torch.rand([1,]).to(device)


net = Transformer(STACKED_NUM, DK, DV, D_MODEL, H, embedding_matrix).to(device)
print(Q.dtype)
o = net(Q, V, Q_fea, K_fea, Q_w, K_w)
# print t
print(o.size())
# print o
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(net))

torch.int64
torch.Size([1, 1])
5649409


In [4]:
# tmp_m = torch.load('../task2/best_loss.pt')
# net.decoder.load_state_dict(tmp_m.decoder.state_dict())
# net.encoder.load_state_dict(tmp_m.encoder.state_dict())
# print 'load weight done'

In [4]:




links = np.genfromtxt(join(data_path,'t3-fake.txt'), dtype=np.int32)
idx_map = {node:idx for idx, node in enumerate(list(set(links.flatten().tolist())))}
N = links.shape[0]
adj_mat = np.zeros([N,N], dtype=np.uint8)
for i in range(links.shape[0]):
    src, dst = links[i].tolist()
    adj_mat[idx_map[src], idx_map[dst]] = 1




In [5]:
import numpy as np
import scipy.sparse as sp
from tqdm import tqdm

def positive_bootsrap_generator(edges, xml_id_map, node_emb_dict):
    num_edge = len(edges)
        
    while True:
        for idx in np.random.permutation(num_edge):
            src, dst = edges[idx, :]
            Q, Q_w = xml_id_map[dst]
            K, K_w = xml_id_map[src]
            Q_fea = node_emb_dict[dst]
            K_fea = node_emb_dict[src]
            yield Q, K, Q_fea, K_fea, Q_w, K_w
def negative_bootsrap_generator(adj_mat, links, idx_map, xml_id_map, training_node_list, node_emb_dict, neighbor_link_rate=0.8):
    
    
    exist_node_list = xml_id_map.keys()
    exist_N = len(training_node_list)
    N = adj_mat.shape[0]
    
#     adj mat
    links = np.array(list(map(idx_map.get, links.flatten())),
                     dtype=np.int32).reshape(links.shape)
    
    adj_sp = sp.coo_matrix((np.ones(links.shape[0]), (links[:, 0], links[:, 1])),
                        shape=(N, N),
                        dtype=np.uint8)
    adj_sp_2 = (sp.coo_matrix.dot(adj_sp,adj_sp) + adj_sp).tocoo()
    
    rev_map = {v:k for k,v in idx_map.items()}
    adj_map = {i:[] for i in range(N)}
    with tqdm(total=len(adj_sp_2.row)) as pbar:
        for i,j,v in zip(adj_sp_2.row, adj_sp_2.col, adj_sp_2.data):
            if adj_mat[i, j] != 1 and v == 1:
                adj_map[i].append(j)
            pbar.update(1)
#             print i,N
                
    while True:
        src = training_node_list[np.random.randint(exist_N)]
        
#         choose neighbor link
        if np.random.rand(1) <= neighbor_link_rate:
        
            i = idx_map[src]
            high = len(adj_map[i])
            while high == 0:
                src = training_node_list[np.random.randint(exist_N)]
                i = idx_map[src]
                high = len(adj_map[i])
                
            idx = np.random.randint(high)
            dst = adj_map[i][idx]
            dst = rev_map[dst]
        else:
            dst = training_node_list[np.random.randint(exist_N)]
            while adj_mat[idx_map[src], idx_map[dst]] == 1:
                dst = training_node_list[np.random.randint(exist_N)]
        Q, Q_w = xml_id_map[dst]
        K, K_w = xml_id_map[src]
        Q_fea = node_emb_dict[dst]
        K_fea = node_emb_dict[src]
        yield Q, K, Q_fea, K_fea, Q_w, K_w

def val_data(edges, xml_id_map):
    Q, K = [],[]
    Q_f, K_f = [],[]
    Q_w, K_w = [],[]
    
    for idx in range(edges.shape[0]):
        src, dst = edges[idx, :]
        q, q_w = xml_id_map[dst]
        k, k_w = xml_id_map[src]
        q_fea = node_emb_dict[dst]
        k_fea = node_emb_dict[src]
        
        Q.append(q)
        K.append(k)
        Q_f.append(q_fea)
        K_f.append(k_fea)
        Q_w.append(q_w)
        K_w.append(k_w)
        
        
    Q = np.vstack(Q)
    K = np.vstack(K)
    Q_fea = np.vstack(Q_f)
    K_fea = np.vstack(K_f)
    Q_w = np.vstack(Q_w)
    K_w = np.vstack(K_w)
    
    return Q, K, Q_fea, K_fea, Q_w, K_w
    
N = links.shape[0]
idx = np.random.permutation(N)
train_idx = idx[N//10:]
val_idx = idx[:N//10]

pos_G = positive_bootsrap_generator(links[train_idx,:], xml_id_map, node_emb_dict)
training_node_list = list(set(links[train_idx,:].flatten().tolist()))
neg_G = negative_bootsrap_generator(adj_mat, links, idx_map, xml_id_map, training_node_list, node_emb_dict)
val_Q, val_K, val_Q_fea, val_K_fea, val_Q_w, val_K_w = val_data(links[val_idx,:], xml_id_map)
q,k,q_f,k_f,q_w,k_w = next(pos_G)
print(q.shape,k.shape, q_f.shape, k_f.shape, q_w.shape, k_w.shape)

q,k,q_f,k_f,q_w,k_w = next(neg_G)
print(q.shape,k.shape, q_f.shape, k_f.shape, q_w.shape, k_w.shape)
print(val_Q.shape,val_K.shape, val_Q_fea.shape, val_K_fea.shape, val_Q_w.shape, val_K_w.shape)


((150,), (150,), (128,), (128,), (1,), (1,))


100%|██████████| 880464/880464 [00:02<00:00, 319909.54it/s]

((150,), (150,), (128,), (128,), (1,), (1,))
((11058, 150), (11058, 150), (11058, 128), (11058, 128), (11058, 1), (11058, 1))


In [ ]:
from collections import deque
import time
def dump_log(model, n_iter, loss, acc, val_loss, val_acc, log_file_stream, tmp_model_path):
    log_text = '%.7d<split>%.5f<split>%.5f<split>%.5f<split>%.5f\n' % (n_iter, loss, acc, val_loss, val_acc)
    log_file_stream.write(log_text)
    if n_iter % 100 == 0 :
        log_file_stream.flush()
        torch.save(model, tmp_model_path)

acc_q = deque(maxlen=10000)
loss_q = deque(maxlen=10000)
val_acc_q = deque(maxlen=10000)
val_loss_q = deque(maxlen=10000)
criterion = nn.BCELoss()
# 
model = net
model.cuda()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)
#
interval = 100
t = time.time()
print 'start training.'
best_acc  = 0
best_loss = float('inf')
for i in range(1,1000000):
    with open('log.txt', 'a') as f:
        
        optimizer.zero_grad()
        model.train()
    #     positive
        q,k,q_f,k_f,q_w,k_w = next(pos_G)
        q,k = torch.LongTensor(q), torch.LongTensor(k)
        q_f,k_f = torch.FloatTensor(q_f), torch.FloatTensor(k_f)
        q_w,k_w = torch.FloatTensor(q_w), torch.FloatTensor(k_w)
        
        output = model(q.cuda(), k.cuda(), q_f.cuda(), k_f.cuda(), q_w.cuda(), k_w.cuda())
        acc = 1 if output.flatten().item() > 0.5 else 0
        acc_q.append(acc)
        pos_loss = criterion(output, torch.FloatTensor([[1]]).cuda() )

#         negative
        q,k,q_f,k_f,q_w,k_w = next(neg_G)
        
        q,k = torch.LongTensor(q), torch.LongTensor(k)
        q_f,k_f = torch.FloatTensor(q_f), torch.FloatTensor(k_f)
        q_w,k_w = torch.FloatTensor(q_w), torch.FloatTensor(k_w)
        
        output = model(q.cuda(), k.cuda(), q_f.cuda(), k_f.cuda(), q_w.cuda(), k_w.cuda())
        
        acc = 1 if output.flatten().item() < 0.5 else 0
        acc_q.append(acc)
        neg_loss = criterion(output, torch.FloatTensor([[0]]).cuda())
        loss = pos_loss + neg_loss
        loss_q.append(loss.item())
        loss.backward()
        optimizer.step()
    #     val
        model.eval()
        with torch.no_grad():
            val_i = i % val_Q.shape[0]
            q,k = val_Q[val_i,:], val_K[val_i,:]
            q_f,k_f = val_Q_fea[val_i,:], val_K_fea[val_i,:]
            q_w,k_w = val_Q_w[val_i,:], val_K_w[val_i,:]
            q,k = torch.LongTensor(q), torch.LongTensor(k)
            q_f,k_f = torch.FloatTensor(q_f), torch.FloatTensor(k_f)
            q_w,k_w = torch.FloatTensor(q_w), torch.FloatTensor(k_w)
        
            output = model(q.cuda(), k.cuda(), q_f.cuda(), k_f.cuda(), q_w.cuda(), k_w.cuda())
            val_acc = 1 if output.flatten().item() > 0.5 else 0
            val_acc_q.append(val_acc)

            val_loss = criterion(output, torch.FloatTensor([[1]]).cuda() )
            val_loss_q.append(val_loss.item())

        model.eval()
        acc = float(np.mean(acc_q))
        loss = float(np.mean(loss_q))
        val_acc = float(np.mean(val_acc_q))
        val_loss = float(np.mean(val_loss_q))

        if i % interval == 0:
            print('iter: {:04d}'.format(i+1),
                  'loss_train: {:.4f}'.format(loss),
                  'acc_train: {:.4f}'.format(acc),
                  'loss_val: {:.4f}'.format(val_loss),
                  'acc_val: {:.4f}'.format(val_acc),
                  'time: {:.4f}s'.format((time.time() - t)))
        if i > 100:
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model, './best_acc.pt')
                with open('./best.txt', 'a') as g:
                    g.write('best acc at %d with %.5f\n' % (i+1, best_acc))

            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(model, './best_loss.pt')
                with open('./best.txt', 'a') as g:
                    g.write('best loss at %d with %.5f\n' % (i+1, best_loss))
            
        dump_log(model, i+1, loss, acc, val_loss, val_acc, f, './tmp.pt')


# Train model
print("Optimization Finished!")
# print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

start training.


/home/k123/env/python2.7.12/local/lib/python2.7/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Transformer. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/k123/env/python2.7.12/local/lib/python2.7/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Word_Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/k123/env/python2.7.12/local/lib/python2.7/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Stack_Encoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/k123/env/python2.7.12/local/lib/python2.7/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Encoder. It won't be checked f

('iter: 0101', 'loss_train: 1.3737', 'acc_train: 0.5300', 'loss_val: 0.6818', 'acc_val: 0.6000', 'time: 19.8478s')
('iter: 0201', 'loss_train: 1.3782', 'acc_train: 0.5200', 'loss_val: 0.6827', 'acc_val: 0.5850', 'time: 39.8184s')
('iter: 0301', 'loss_train: 1.3742', 'acc_train: 0.5333', 'loss_val: 0.6866', 'acc_val: 0.5533', 'time: 59.6018s')
('iter: 0401', 'loss_train: 1.3659', 'acc_train: 0.5500', 'loss_val: 0.6788', 'acc_val: 0.5925', 'time: 80.2137s')
('iter: 0501', 'loss_train: 1.3675', 'acc_train: 0.5400', 'loss_val: 0.6766', 'acc_val: 0.5920', 'time: 101.4449s')
('iter: 0601', 'loss_train: 1.3658', 'acc_train: 0.5475', 'loss_val: 0.6770', 'acc_val: 0.6033', 'time: 124.9052s')
('iter: 0701', 'loss_train: 1.3614', 'acc_train: 0.5536', 'loss_val: 0.6770', 'acc_val: 0.6057', 'time: 144.6061s')
('iter: 0801', 'loss_train: 1.3600', 'acc_train: 0.5550', 'loss_val: 0.6825', 'acc_val: 0.5913', 'time: 164.4921s')
('iter: 0901', 'loss_train: 1.3611', 'acc_train: 0.5589', 'loss_val: 0.6814'

('iter: 7801', 'loss_train: 1.3228', 'acc_train: 0.6167', 'loss_val: 0.6574', 'acc_val: 0.6659', 'time: 1678.1083s')
('iter: 7901', 'loss_train: 1.3225', 'acc_train: 0.6172', 'loss_val: 0.6566', 'acc_val: 0.6665', 'time: 1704.9600s')
('iter: 8001', 'loss_train: 1.3230', 'acc_train: 0.6159', 'loss_val: 0.6567', 'acc_val: 0.6666', 'time: 1725.0678s')
('iter: 8101', 'loss_train: 1.3224', 'acc_train: 0.6174', 'loss_val: 0.6568', 'acc_val: 0.6669', 'time: 1748.1393s')
('iter: 8201', 'loss_train: 1.3216', 'acc_train: 0.6172', 'loss_val: 0.6562', 'acc_val: 0.6678', 'time: 1772.1571s')
('iter: 8301', 'loss_train: 1.3223', 'acc_train: 0.6182', 'loss_val: 0.6564', 'acc_val: 0.6672', 'time: 1795.1672s')
('iter: 8401', 'loss_train: 1.3230', 'acc_train: 0.6173', 'loss_val: 0.6561', 'acc_val: 0.6670', 'time: 1814.9661s')
('iter: 8501', 'loss_train: 1.3230', 'acc_train: 0.6182', 'loss_val: 0.6557', 'acc_val: 0.6681', 'time: 1836.4558s')
('iter: 8601', 'loss_train: 1.3233', 'acc_train: 0.6177', 'loss_

('iter: 14801', 'loss_train: 1.2987', 'acc_train: 0.6376', 'loss_val: 0.6443', 'acc_val: 0.6769', 'time: 3184.9736s')
('iter: 14901', 'loss_train: 1.2988', 'acc_train: 0.6388', 'loss_val: 0.6436', 'acc_val: 0.6772', 'time: 3206.0012s')
('iter: 15001', 'loss_train: 1.2981', 'acc_train: 0.6381', 'loss_val: 0.6432', 'acc_val: 0.6770', 'time: 3227.2217s')
('iter: 15101', 'loss_train: 1.2981', 'acc_train: 0.6385', 'loss_val: 0.6435', 'acc_val: 0.6769', 'time: 3247.2020s')
('iter: 15201', 'loss_train: 1.2977', 'acc_train: 0.6381', 'loss_val: 0.6439', 'acc_val: 0.6760', 'time: 3267.1077s')
('iter: 15301', 'loss_train: 1.2984', 'acc_train: 0.6379', 'loss_val: 0.6445', 'acc_val: 0.6751', 'time: 3286.9840s')
('iter: 15401', 'loss_train: 1.2980', 'acc_train: 0.6391', 'loss_val: 0.6442', 'acc_val: 0.6752', 'time: 3306.8589s')
('iter: 15501', 'loss_train: 1.2987', 'acc_train: 0.6392', 'loss_val: 0.6442', 'acc_val: 0.6746', 'time: 3326.7503s')
('iter: 15601', 'loss_train: 1.2980', 'acc_train: 0.6388

('iter: 22301', 'loss_train: 1.2688', 'acc_train: 0.6552', 'loss_val: 0.6333', 'acc_val: 0.6552', 'time: 4730.6681s')
('iter: 22401', 'loss_train: 1.2680', 'acc_train: 0.6580', 'loss_val: 0.6328', 'acc_val: 0.6555', 'time: 4754.0004s')
('iter: 22501', 'loss_train: 1.2669', 'acc_train: 0.6580', 'loss_val: 0.6327', 'acc_val: 0.6557', 'time: 4774.0279s')
('iter: 22601', 'loss_train: 1.2659', 'acc_train: 0.6579', 'loss_val: 0.6307', 'acc_val: 0.6567', 'time: 4799.1397s')
('iter: 22701', 'loss_train: 1.2660', 'acc_train: 0.6586', 'loss_val: 0.6302', 'acc_val: 0.6566', 'time: 4822.2062s')
('iter: 22801', 'loss_train: 1.2655', 'acc_train: 0.6582', 'loss_val: 0.6307', 'acc_val: 0.6558', 'time: 4842.2470s')
('iter: 22901', 'loss_train: 1.2652', 'acc_train: 0.6583', 'loss_val: 0.6312', 'acc_val: 0.6546', 'time: 4862.2176s')
('iter: 23001', 'loss_train: 1.2642', 'acc_train: 0.6583', 'loss_val: 0.6315', 'acc_val: 0.6537', 'time: 4882.1784s')
('iter: 23101', 'loss_train: 1.2632', 'acc_train: 0.6586

('iter: 29301', 'loss_train: 1.2413', 'acc_train: 0.6630', 'loss_val: 0.6200', 'acc_val: 0.6513', 'time: 6288.3982s')
('iter: 29401', 'loss_train: 1.2413', 'acc_train: 0.6619', 'loss_val: 0.6188', 'acc_val: 0.6523', 'time: 6318.6438s')
('iter: 29501', 'loss_train: 1.2398', 'acc_train: 0.6623', 'loss_val: 0.6189', 'acc_val: 0.6517', 'time: 6349.2229s')
('iter: 29601', 'loss_train: 1.2387', 'acc_train: 0.6649', 'loss_val: 0.6190', 'acc_val: 0.6511', 'time: 6378.4489s')
('iter: 29701', 'loss_train: 1.2379', 'acc_train: 0.6657', 'loss_val: 0.6185', 'acc_val: 0.6514', 'time: 6407.9489s')
('iter: 29801', 'loss_train: 1.2376', 'acc_train: 0.6664', 'loss_val: 0.6188', 'acc_val: 0.6504', 'time: 6437.2298s')
('iter: 29901', 'loss_train: 1.2367', 'acc_train: 0.6659', 'loss_val: 0.6183', 'acc_val: 0.6504', 'time: 6466.6216s')
('iter: 30001', 'loss_train: 1.2375', 'acc_train: 0.6658', 'loss_val: 0.6184', 'acc_val: 0.6507', 'time: 6496.5610s')
('iter: 30101', 'loss_train: 1.2370', 'acc_train: 0.6651

('iter: 36301', 'loss_train: 1.2110', 'acc_train: 0.6757', 'loss_val: 0.6102', 'acc_val: 0.6606', 'time: 8165.3133s')
('iter: 36401', 'loss_train: 1.2109', 'acc_train: 0.6763', 'loss_val: 0.6102', 'acc_val: 0.6607', 'time: 8189.6493s')
('iter: 36501', 'loss_train: 1.2114', 'acc_train: 0.6750', 'loss_val: 0.6094', 'acc_val: 0.6618', 'time: 8214.0180s')
('iter: 36601', 'loss_train: 1.2104', 'acc_train: 0.6759', 'loss_val: 0.6086', 'acc_val: 0.6633', 'time: 8238.4001s')
('iter: 36701', 'loss_train: 1.2095', 'acc_train: 0.6768', 'loss_val: 0.6075', 'acc_val: 0.6639', 'time: 8263.0336s')
('iter: 36801', 'loss_train: 1.2090', 'acc_train: 0.6775', 'loss_val: 0.6069', 'acc_val: 0.6650', 'time: 8289.7904s')
('iter: 36901', 'loss_train: 1.2088', 'acc_train: 0.6794', 'loss_val: 0.6072', 'acc_val: 0.6644', 'time: 8314.0491s')
('iter: 37001', 'loss_train: 1.2075', 'acc_train: 0.6798', 'loss_val: 0.6054', 'acc_val: 0.6660', 'time: 8341.4827s')
('iter: 37101', 'loss_train: 1.2072', 'acc_train: 0.6789